# Julia *gotchas* and how to handle them
(Inspired by http://www.stochasticlifestyle.com/7-julia-gotchas-handle/ by Chris Rackauckas.)

**One can write terribly slow code in any language, including Julia.**

Below we address common performance *gotchas* in Julia code.

# Gotcha 1: Global scope

In [ ]:
a=2.0
b=3.0
function linearcombo()
  return 2a+b
end
answer = linearcombo()

@show answer;

The issue here is that the REPL/global scope does not guarantee that `a` and `b` are of a certain type.

In [ ]:
using BenchmarkTools

@btime linearcombo();

This code should never allocate and take 40 ns....

In [ ]:
@code_llvm debuginfo=:none linearcombo()

### How to identify and avoid this issue?

One way to identify the issue is [Traceur.jl](https://github.com/MikeInnes/Traceur.jl). It is basically a codified version of the [performance tips](https://docs.julialang.org/en/v0.6.4/manual/performance-tips/#man-performance-tips-1) in the Julia documentation.

In [ ]:
using Traceur
@trace linearcombo()

#### 1) Wrap code in functions.

In [ ]:
function outer()
    a=2.0
    b=3.0
    function linearcombo()
      return 2a+b
    end
    return linearcombo() 
end

answer = outer()

@show answer;

In [ ]:
@code_llvm debuginfo=:none outer()

This is fast.

In fact, it's not just fast, but as fast as it can be! Julia has figured out the result of the calculation at compile-time and returns **just the result (a literal)!**

(Effectively, `outer() = 7` at run-time.)

In [ ]:
@trace outer()

In [ ]:
@btime outer();

#### 2) Declare globals as (compile-time) constants.

In [ ]:
const A=2.0
const B=3.0

function Linearcombo()
  return 2A+B
end
answer = Linearcombo()

@show answer;

In [ ]:
@code_llvm debuginfo=:none Linearcombo()

In [ ]:
@trace Linearcombo()

In [ ]:
@btime Linearcombo();

Note that the constants above are only compile-time constants, which can be modified:

In [ ]:
const A=1.0

In [ ]:
Linearcombo() # still returns 7, not 5

**3) If the values of `a` and `b` may vary, provide them as function arguments.**

This way, the compiler can compile specializations for all possible input types!

In [ ]:
a=2.0
b=3.0

function lincombo(a,b)
    return 2a+b
end

answer = lincombo(a,b)

@show answer;

In [ ]:
@code_llvm debuginfo=:none lincombo(a,b)

In [ ]:
@btime lincombo($a,$b);

#### Take home message: Always wrap a performance critical piece of code in a self-contained function.

# Gotcha 2: Type-instabilities

What's bad for performance about the following function?

In [ ]:
function g()
  x=1
  for i = 1:10
    x = x/2
  end
  return x
end

In [ ]:
@code_llvm debuginfo=:none g()

A more drastic example

In [ ]:
f() = rand([1.0, 2, "3"])

In [ ]:
@code_llvm debuginfo=:none f()

In [ ]:
@btime g();

### How to find and deal with type-instabilities

#### 1) Avoid type changes

Initialize `x` as `Float64` and it's fast.

In [ ]:
function h()
  x=1.0
  for i = 1:10
    x = x/2
  end
  return x
end

In [ ]:
@code_llvm debuginfo=:none h()

In [ ]:
@btime h();

#### 2) Detect issues with `@code_warntype` (or `@trace`)

In [ ]:
@code_warntype g()

(On a side note: since the type can only vary between `Float64` and `Int64`, Julia can still produce reasonable code by *union splitting*. See the blog post by Tim Holy: https://julialang.org/blog/2018/08/union-splitting)

In [ ]:
@code_warntype h()

#### 3) The C/Fortran way: specify types (to get errors or to heal the problem by conversion)

In [ ]:
function g2()
  x::Int64 = 1
  for i = 1:10
    x = x/2
  end
  return x
end

In [ ]:
g2()

In [ ]:
function g3()
  x::Float64 = 1 # triggers an implicit conversion to Float64
  for i = 1:10
    x = x/2
  end
  return x
end

In [ ]:
@code_llvm debuginfo=:none g3()

#### 4) Function barriers

In [ ]:
data = Union{Int64,Float64,String}[4, 2.0, "test", 3.2, 1]

In [ ]:
function calc_square(x)
  for i in eachindex(x)
    val = x[i]
    val^2
  end
end

In [ ]:
@code_warntype calc_square(data)

In [ ]:
function calc_square_outer(x)
  for i in eachindex(x)
    calc_square_inner(x[i])
  end
end

calc_square_inner(x) = x^2

In [ ]:
@code_warntype calc_square_inner(data[1])

#### Comments:

Why allow type-instabilities in the first place? Convenience vs performance tradeoff.

Note that type instabilities can naturally occur (reading files, user input etc.) so not any red marker is bad/avoidable.

#### Take home message: watch out for type-instabilities in performance critical parts of your code.

# Gotcha 3: Temporary allocations in vectorized code

In [ ]:
function f()
  x = [1.0,5.0,6.0]
  for i in 1:100_000
    x = x + 2*x
  end
  return x
end

In [ ]:
@btime f();

### How to handle it? → More dots or more explicity

(Great blog post by Steven G. Johnson: https://julialang.org/blog/2017/01/moredots ([related notebook](https://github.com/JuliaLang/www.julialang.org/blob/master/blog/_posts/moredots/More-Dots.ipynb)))

In [ ]:
function f()
    x = [1.0,5.0,6.0]
    for i in 1:100_000    
        for k in 1:3
            x[k] = x[k] + 2 * x[k]
        end
    end
    return x
end
@btime f();

In [ ]:
function f()
    x = [1.0,5.0,6.0]
    for i in 1:100_000
        x = x .+ 2 .* x
    end
    return x
end
@btime f();

In [ ]:
function f()
    x = [1.0,5.0,6.0]
    for i in 1:100_000
        x .= x .+ 2 .* x
    end
    return x
end
@btime f();

In [ ]:
function f()
    x = [1.0,5.0,6.0]
    for i in 1:100_000
        @. x = x + 2*x
    end
    return x
end
@btime f();

### Extra performance: `@inbounds`

In [ ]:
function f()
    x = [1.0,5.0,6.0]
    @inbounds for i in 1:100_000    
        for k in 1:3
            x[k] = x[k] + 2*x[k]
        end
    end
    return x
end
@btime f();

# Gotcha 4: Unnecessary copies

Assume you were asked to compute the sum of a slice of a matrix, say, the first column. Naively, you'd probably do something like this:

In [ ]:
M = rand(3,3);

In [ ]:
sum(M[1:3,1])

In [ ]:
f(x,M) = sum(M[1:3,1])

@btime f($x,$M);

Hm, there is an allocation?! Just to sum up elements? The thing is that **slices by default create copies!**

In [ ]:
X = M[1:3,1] # copy of the data in M

In [ ]:
X[1] = 42

In [ ]:
M

To avoid the copy, we can use `@view`/`view`:

In [ ]:
X = @view M[1:3,1] # equivalent to view(M, 1:3, 1)

In [ ]:
X[1] = 42

In [ ]:
M

In [ ]:
sum(@view M[1:3,1])

In [ ]:
g(x,M) = sum(@view M[1:3,1])

@btime g($x, $M);

To avoid multiple `@view`/`view` commands in a line of code there is `@views`:

In [ ]:
@views M[1:3,1] .+ M[1:3,2]

# Gotcha 5: Abstract fields

In [ ]:
struct MyType
    x::AbstractFloat
    y::AbstractString
end

f(a::MyType) = a.x^2 + sqrt(a.x)

In [ ]:
a = MyType(3.0, "test")

@btime f($a);

In [ ]:
struct MyTypeConcrete
    x::Float64
    y::String
end

f(b::MyTypeConcrete) = b.x^2 + sqrt(b.x)

In [ ]:
b = MyTypeConcrete(3.0, "test")

@btime f($b);

Note that the latter implementation is **more than 30x faster**!

### How to handle it?

But what if I want to accept any kind of `AbstractFloat` and `AbstractString` in my type?

Use type parameters!

In [ ]:
struct MyTypeParametric{A<:AbstractFloat, B<:AbstractString}
    x::A
    y::B
end

f(c::MyTypeParametric) = c.x^2 + sqrt(c.x)

In [ ]:
c = MyTypeParametric(3.0, "test")

From the type alone the compiler knows what the structure contains and can produce optimal code:

In [ ]:
@btime f($c);

In [ ]:
c = MyTypeParametric(Float32(3.0), SubString("test"))

In [ ]:
@btime f($c);

# Gotcha 6: Column major order

<img src="imgs/column-major-2D.png">
(<a href=https://mitmath.github.io/18337/lecture2/optimizing>Image source</a>)

In [ ]:
M = rand(1000,1000);

function fcol(M)
    for col in 1:size(M, 2)
        for row in 1:size(M, 1)
            M[row, col] = 42
        end
    end
    nothing
end

function frow(M)
    for row in 1:size(M, 1)
        for col in 1:size(M, 2)
            M[row, col] = 42
        end
    end
    nothing
end

In [ ]:
@btime fcol($M)

In [ ]:
@btime frow($M)

#### Take home message: fastest varying index goes first!

# Core messages of this Notebook

* Gotcha 1: **Wrap code in self-contained functions** in performance critical applications, i.e. avoid global scope.
* Gotcha 2: Write **type-stable code** (check with `@code_warntype`).
* Gotcha 3: Use **broadcasting (more dots)** to avoid temporary allocations in vectorized code (or write out loops).
* Gotcha 4: Use **views** instead of copies to avoid unnecessary allocations.
* Gotcha 5: **Types should always have concrete fields.** If you don't know them in advance, use type parameters.
* Gotcha 6: Be aware of **column major order** when looping over arrays.

Want to read more about optimizing serial Julia code? Check out <a href=https://mitmath.github.io/18337/lecture2/optimizing>this MIT lecture</a>.